In [1]:
import sys

sys.path.append('../')

import numpy as np
import gym

import or_suite
import pickle

In [2]:
algorithm = 'Stable'
problem = 'beta'
alpha = '1'
num_ambulance = '1'
dir_path = '../data/ambulance_metric_%s_%s_%s_%s'%(algorithm,num_ambulance, alpha, problem)
dir_path = dir_path+'/trajectory.obj'

In [3]:
print(dir_path)

../data/ambulance_metric_Stable_1_1_beta/trajectory.obj


In [4]:
with open(dir_path, 'rb') as f:
    x = pickle.load(f)


In [5]:
x

[{'iter': 0,
  'episode': 1,
  'step': 0,
  'oldState': array([0]),
  'action': array([0]),
  'reward': -0.0,
  'newState': array([0.9017086], dtype=float32),
  'info': {'arrival': 0.9017086254601233}},
 {'iter': 0,
  'episode': 1,
  'step': 1,
  'oldState': array([0.9017086], dtype=float32),
  'action': array([0.9017086], dtype=float32),
  'reward': -0.0,
  'newState': array([0.8516732], dtype=float32),
  'info': {'arrival': 0.8516731965445568}},
 {'iter': 0,
  'episode': 1,
  'step': 2,
  'oldState': array([0.8516732], dtype=float32),
  'action': array([0.8516732], dtype=float32),
  'reward': -0.0,
  'newState': array([0.91692984], dtype=float32),
  'info': {'arrival': 0.9169298378289636}},
 {'iter': 0,
  'episode': 1,
  'step': 3,
  'oldState': array([0.91692984], dtype=float32),
  'action': array([0.91692984], dtype=float32),
  'reward': -0.0,
  'newState': array([0.98049366], dtype=float32),
  'info': {'arrival': 0.9804936830220771}},
 {'iter': 0,
  'episode': 1,
  'step': 4,
  'o

{'iter': 0,
 'episode': 1,
 'step': 0,
 'oldState': array([0]),
 'action': array([0]),
 'reward': -0.0,
 'newState': array([0.9017086], dtype=float32),
 'info': {'arrival': 0.9017086254601233}}

In [30]:
dist = lambda x,y: np.abs(x-y)

In [29]:
def mean_response_time(traj, dist):
    actions = []
    info = []
    mrt = 0
    for i in range(len(x)):
        data = x[i]
        # print(data['action'])
        # print(data['info'])
        mrt += np.min(dist(data['action'],data['info']['arrival']))
    return mrt / len(x)

In [28]:
mean_response_time(x, dist)

0.29062895311674675

In [6]:
# TODO:
#    Figure how to go from dataframe of a trajectory to calculating the fairness metrics

In [11]:
epLen = 3 #changes depending on environment config
def mean_online_allocation(traj, epLen=3):
    """
    Computes the mean allocation given the trajectory of the online algorithm over arbitrary episodes
    
    Input: 
        - traj: trajectory of algorithm, all actions taken at every state
        -epLen: length of an episode
    Returns:
        - X: Function i -> mean allocation given at location i
        - N: Mean endowment at location i
    """
    X = [np.zeros(traj[0]['action'].shape) for i in range(epLen)]
    count = [0 for i in range(epLen)]
    N = [np.zeros(traj[0]['info']['type'].shape) for i in range(epLen)]
    for dicto in traj:
        for i in range(epLen):
            if dicto['step'] != i:
                continue
            allocation = dicto['action']
            endowment = dicto['info']['type']
            X[i] += allocation
            N[i] += endowment
            count[i] += 1

    X_final = [X[i]/count[i] for i in range(epLen)]
    N_final = [N[i]/count[i] for i in range(epLen)]
    return X_final, N_final  

In [13]:
X_alg,N = mean_online_allocation(x)
print(X_alg)
print(N)

[array([[16.61789326, 16.61789326],
       [16.96225619, 16.96225619],
       [16.85472784, 16.85472784]]), array([[16.42189223, 16.42189223],
       [16.70840763, 16.70840763],
       [16.17621037, 16.17621037]]), array([[17.00581198, 17.00581198],
       [16.76421348, 16.76421348],
       [16.48858702, 16.48858702]])]
[array([23.576, 25.024, 25.496]), array([25.792, 23.532, 22.464]), array([24.648, 23.56 , 24.628])]


In [ ]:
def offline_allocation(N, env_config):
    import cvxpy as cp
    # Create a 3D k-by-m-by-n variable.
    X_opt = {}
    for i in range(env_config['num_rounds']):
      X_opt[i] = cp.Variable((env_config['weight_matrix'].shape[0],len(env_config['init_budget'])))
    N = cp.Parameter((env_config['weight_matrix'].shape[0],len(env_config['init_budget'])))
    prob = cp.Problem(cp.Minimize(c.T@x),
                     [A @ x <= b])
    prob.solve()

    # Print result.
    print("\nThe optimal value is", prob.value)
    print("A solution x is")
    print(x.value)
    print("A dual solution is")
    print(prob.constraints[0].dual_value)
    
    

In [14]:
def delta_envy(X_alg,X_opt):
    supnorm = lambda v: max([abs[i] for i in v])
    max_envy=0
    for i,allocation in enumerate(X_alg):
        for j,row in enumerate(allocation):
            tmp = supnorm(row-X_opt[i][j,:])
            if tmp >= max_envy:
                max_envy = tmp
    return max_envy

In [16]:
def delta_efficiency(X_alg, env_config):
    B = env_config['init_budget']
    num_types = env_config['weight_matrix'].shape[0]
    return sum([B-sum([X_alg[i][theta,:] for theta in range(num_types)]) for i in range(len(X_alg))])